In [1]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.io
import pandas as pd
import tensorflow as tf
from tensorflow import keras
#import keras
from sklearn.metrics import classification_report
import fgsm

In [2]:
paviaU = scipy.io.loadmat('./PaviaU.mat')['paviaU']
paviaU_gt = scipy.io.loadmat('./PaviaU_gt.mat')['paviaU_gt']

In [3]:
Nmax = paviaU.max()
paviaU = (paviaU/Nmax).astype(np.float32)

In [4]:
paviaU = paviaU.reshape(610*340, 103)

In [5]:
#from sklearn.decomposition import PCA
#pca = PCA(20)
#paviaU_PCA = pca.fit_transform(paviaU)

In [6]:
paviaU_gt = paviaU_gt.reshape(610*340)

In [39]:
X = paviaU.reshape((610*340, 103,1))
y = paviaU_gt
idx = np.random.permutation(len(X))
X = X[idx]
y = y[idx]
nsplit = int(0.2*len(paviaU))
X_train = X[:nsplit]
y_train = y[:nsplit]
X_val = X[nsplit:nsplit*2]
y_val = y[nsplit:nsplit*2]
X_test = X[nsplit*2:]
y_test = y[nsplit*2:]
X,y = None,None

In [40]:
y_train_encode = keras.utils.to_categorical(y_train)
y_val_encode = keras.utils.to_categorical(y_val)
y_test_encode = keras.utils.to_categorical(y_test)


In [41]:
def nnmodel(input_shape):
    X_input = keras.layers.Input((input_shape))
    X = keras.layers.Flatten()(X_input)
    X = keras.layers.Dense(512,activation='relu')(X)
    X = keras.layers.Dense(512,activation='relu')(X)
    X = keras.layers.Dense(512,activation='relu')(X)
    X = keras.layers.Dense(10,activation='softmax')(X)
    model = keras.models.Model(inputs=X_input, outputs=X, name='model')
    return model

In [47]:
mymodel = nnmodel(X_train[0].shape)
mymodel.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
mymodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 103, 1)]          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 103)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               53248     
_________________________________________________________________
dense_13 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_14 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 583,690
Trainable params: 583,690
Non-trainable params: 0
_______________________________________________________

In [48]:
mymodel.fit(X_train,y_train_encode,batch_size=32,epochs=10,validation_data=(X_val,y_val_encode))

Train on 41480 samples, validate on 41480 samples
Epoch 1/10
41480/41480 [==============================] - 8s 195us/sample - loss: 0.6688 - accuracy: 0.7937 - val_loss: 0.5981 - val_accuracy: 0.7911
Epoch 2/10
41480/41480 [==============================] - 7s 179us/sample - loss: 0.5873 - accuracy: 0.7946 - val_loss: 0.5732 - val_accuracy: 0.7934
Epoch 3/10
41480/41480 [==============================] - 7s 180us/sample - loss: 0.5636 - accuracy: 0.7944 - val_loss: 0.5495 - val_accuracy: 0.7937
Epoch 4/10
41480/41480 [==============================] - 7s 175us/sample - loss: 0.5388 - accuracy: 0.7961 - val_loss: 0.5570 - val_accuracy: 0.7921
Epoch 5/10
41480/41480 [==============================] - 7s 177us/sample - loss: 0.5229 - accuracy: 0.7980 - val_loss: 0.5164 - val_accuracy: 0.8006
Epoch 6/10
41480/41480 [==============================] - 7s 176us/sample - loss: 0.5070 - accuracy: 0.8008 - val_loss: 0.5039 - val_accuracy: 0.8018
Epoch 7/10
41480/41480 [==========================

In [49]:
his = mymodel.history.history
plt.plot(his['acc'])
plt.plot(his['val_acc'])

KeyError: 'acc'

In [50]:
mymodel.evaluate(X_test,y_test_encode)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[0.46445526968854217, 0.8128335]

In [51]:
pred = mymodel.predict(X_test)
pred = np.argmax(pred,axis=1)

In [52]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.83      0.97      0.89     98953
           1       0.70      0.07      0.12      3966
           2       0.64      0.36      0.46     11016
           3       0.00      0.00      0.00      1265
           4       0.41      0.07      0.12      1816
           5       0.79      0.69      0.74       815
           6       0.00      0.00      0.00      3004
           7       0.00      0.00      0.00       818
           8       0.23      0.00      0.00      2237
           9       0.41      0.56      0.47       550

    accuracy                           0.81    124440
   macro avg       0.40      0.27      0.28    124440
weighted avg       0.75      0.81      0.76    124440



C:\Users\Lay\Anaconda3\envs\tf2\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
import foolbox

ModuleNotFoundError: No module named 'foolbox'

In [62]:
fmodel = foolbox.models.KerasModel(mymodel,bounds=(0,1))

In [63]:
attack = foolbox.v1.attacks.FGSM(fmodel)

In [81]:
np.array([1]).shape

(1,)

In [71]:
y_val.shape

(41480,)

In [53]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
def create_adversarial_pattern(input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = mymodel(input_image)
        loss = loss_object(input_label, prediction)

      # Get the gradients of the loss w.r.t to the input image.
        gradient = tape.gradient(loss, input_image)
      # Get the sign of the gradients to create the perturbation
        signed_grad = tf.sign(gradient)
    return signed_grad

In [54]:
ak = create_adversarial_pattern(tf.convert_to_tensor(X_val),tf.convert_to_tensor(y_val_encode))

In [56]:
valak = X_val+0.001*ak

In [57]:
pp = mymodel.predict(valak)

In [58]:
pp = np.argmax(pp,axis=1)

In [61]:
mymodel.evaluate(valak,y_val_encode)

41480/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.5505473363465605, 0.7830521]